In [1]:
import os

import pandas as pd

import numpy as np

In [2]:
# Déclaration des constantes
DATA_DIR = '../data/external/'
PROCESS_DIR = '../data/processed/'

CITIES_FILENAME = 'base-cc-filosofi-2015.xls'
COMMUNES_FILENAME = 'PS_communes.csv'
ANALYSIS_FILENAME = 'analyse_filosofi.csv'
GEO_FILENAME = 'villes_france.csv'

In [3]:
filo_df = pd.read_excel(os.path.join(DATA_DIR, CITIES_FILENAME), skiprows=5)
filo_df.head()

,CODGEO,LIBGEO,NBMENFISC15,NBPERSMENFISC15,MED15,PIMP15,TP6015,TP60AGE115,TP60AGE215,TP60AGE315,...,PPEN15,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15,D115,D915,RD15
0,01001,L'Abergement-Clémenciat,308.0,801.0,22228.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,100.0,245.5,22883.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01004,Ambérieu-en-Bugey,6216.0,13917.0,19735.200000,53.312962,17.227132,19.058154,20.856552,22.513401,...,26.9,9.5,6.7,2.7,2.0,2.0,-15.8,10398.571429,33801.2,3.250562
3,01005,Ambérieux-en-Dombes,624.0,1671.5,23182.666667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01006,Ambléon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
filo_df.query('LIBGEO == "Paris"') #total

,CODGEO,LIBGEO,NBMENFISC15,NBPERSMENFISC15,MED15,PIMP15,TP6015,TP60AGE115,TP60AGE215,TP60AGE315,...,PPEN15,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15,D115,D915,RD15
30081,75056,Paris,1030953.0,2096399.0,26430.9,70.185952,16.157611,17.404165,13.522157,18.062459,...,20.0,18.4,2.9,0.9,1.0,0.9,-27.4,9864.319444,63781.0,6.465829


In [5]:
print('# foyers fiscaux NaN: ',filo_df.NBPERSMENFISC15.isnull().sum())

# foyers fiscaux NaN:  3596


In [6]:
print('Shape filisofi: ',filo_df.shape)

Shape filisofi:  (35848, 29)


In [7]:
com_df = pd.read_csv(os.path.join(PROCESS_DIR, COMMUNES_FILENAME),sep=';')
com_df.head()

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,organe,radiologiste,reeducateur_podologe
0,01001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,01002,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01004,1.0,4.0,21.0,35.0,9.0,15.0,25.0,8.0,32.0,0.0,5.0,36.0
3,01005,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,5.0
4,01007,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,3.0


In [8]:
com_df.query('codecommuneetablissement == "75056"') 

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,organe,radiologiste,reeducateur_podologe


Paris global est absent de cette table, mais nous avons les arrondissements !

In [9]:
com_df.query('codecommuneetablissement == "75101"') 

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,organe,radiologiste,reeducateur_podologe
16894,75101,0.0,5.0,86.0,75.0,4.0,85.0,87.0,3.0,41.0,1.0,9.0,110.0


## Nous créons de nouvelles variables:
- Somme de toutes les données médicales
- 3 groupes (selon la DREES) - Adapté à nos données
    - accès au soin de base: peu ou pas de médecins généralistes, hopitaux, pharmacies
    - accès aux spécialistes réguliers: chirurgiens-dentistes, gynécologues, ophtalmologues, pédiatres, infirmiers, masseurs-kinésithérapeutes...
    - accès aux spécialistes vraiment particuliers: (cardiologie, rhumatologie, pneumologie, diabétologie, cancérologie, neurologie... mais à adapter à nos données disponibles


In [10]:
#Somme de tous les services médicaux par commune
com_df['services_medicaux'] = com_df.iloc[:,1:].sum(axis=1)

com_df['soin_de_base'] = com_df['generaliste'] + com_df['hopital']

com_df['spe_regulier'] = com_df['chirurgien'] + com_df['dentiste'] + com_df['infirmiers'] +  com_df['radiologiste'] + com_df['reeducateur_podologe'] + com_df['analyse_medicale'] + com_df['ambulance'] 

com_df['spe_particulier'] = com_df['autre'] + com_df['autre_specialiste'] + com_df['organe'] 

In [11]:
com_df['soin_de_base'].describe()

count    20478.000000
mean         5.422795
std         28.561624
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max       1202.000000
Name: soin_de_base, dtype: float64

In [12]:
com_df['spe_regulier'].describe()

count    20478.000000
mean        16.726633
std         73.911477
min          0.000000
25%          1.000000
50%          3.000000
75%         11.000000
max       3424.000000
Name: spe_regulier, dtype: float64

In [13]:
com_df['spe_particulier'].describe()

count    20478.000000
mean         6.247632
std         40.998998
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max       1951.000000
Name: spe_particulier, dtype: float64

In [14]:
com_df.head()

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,organe,radiologiste,reeducateur_podologe,services_medicaux,soin_de_base,spe_regulier,spe_particulier
0,01001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,01002,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,01004,1.0,4.0,21.0,35.0,9.0,15.0,25.0,8.0,32.0,0.0,5.0,36.0,191.0,33.0,102.0,56.0
3,01005,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,5.0,11.0,2.0,9.0,0.0
4,01007,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,3.0,15.0,5.0,7.0,3.0


## Merge 

In [15]:
df = (com_df.merge(filo_df, left_on = 'codecommuneetablissement', right_on = 'CODGEO', indicator=True,
                   how='outer')
      .fillna(value={c: 0 for c in com_df.columns if c is not 'codecommuneetablissement'}))

In [16]:
df._merge.value_counts()

both          20404
right_only    15444
left_only        74
Name: _merge, dtype: int64

In [17]:
df.head()

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,...,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15,D115,D915,RD15,_merge
0,01001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,01002,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,01004,1.0,4.0,21.0,35.0,9.0,15.0,25.0,8.0,32.0,...,9.5,6.7,2.7,2.0,2.0,-15.8,10398.571429,33801.200000,3.250562,both
3,01005,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,01007,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,4.0,...,8.1,4.0,2.4,0.8,0.8,-15.1,13620.666667,34606.923077,2.540766,both


In [18]:
print('Shape full dataframe: ',df.shape)

Shape full dataframe:  (35922, 47)


## Ajout de données géographiques   
https://sql.sh/736-base-donnees-villes-francaises?fbclid=IwAR2zcznQihOy0UyFuQfGwlTV8EcWVLBd5CRM8OSt_UkqI-yad-6JvdsLkqo

**Département** : numéro du département.  
**Slug** : identifiant unique en minuscule, sans accent et sans espace. Peut servir pour faire les URLs d’un site web.  
**Nom** : nom en majuscule et sans accent.  
**Nom simple** : nom en minuscule, sans accent et avec les tirets remplacés par des espaces. Peut être utilisé pour faire une recherche lorsqu’on ne sait pas si le nom de ville possède un tiret ou des espaces (ex : “Saint-Étienne” possède un tiret comme séparateur, tandis que “Le Havre” possède un espace comme séparateur)  
**Nom reel** : nom correct avec les accents  
**Nom soundex** : soundex du nom de la ville (permet de trouver des villes qui se prononcent presque pareil) [ajouté le 31/10/2013]  
**Nom metaphone** : metaphone du nom de la ville (même utilité que le soundex) [ajouté le 31/10/2013]
Code postal : code postal de la ville. Si la ville possède plusieurs code postaux, ils sont tous listés et séparés par un tiret [ajouté le 31/10/2013]  
**Numéro de commune** : numéro de la commune dans le département. Combiné avec le numéro de département, il permet de créer le code INSEE sous 5 caractères.  
**Code commune (ou code INSEE)** : identifiant unique sous 5 caractères  
**Arrondissement** : arrondissement de la ville   
**Canton** : canton de la ville  
**Population en 2010** : nombre d’habitants lors du recensement de 2010  
**Population en 1999** : nombre d’habitants lors du recensement de 1999  
**Population en 2012** (approximatif) : valeur exprimée en centaine  
**Densité en 2010** : nombre d’habitants au kilomètre carré arrondie à l’entier. Calculé à partir du nombre d’habitant et de la surface de la ville [corrigé le 02/07/2014]  
**Surface / superficie** : surface de la ville en kilomètre carrée [corrigé le 02/07/2014]  
**Longitude/latitude en degré** : géolocalisation du centre de la ville. Permet de localiser la ville sur une carte (exemple : carte Google Maps) [ajouté le 31/10/2013, corrigé le 07/11/2013]   
**Longitude/latitude en GRD** : géolocalisation exprimée en GRD  
**Longitude/latitude en DMS** (Degré Minute Seconde) : géolocalisation exprimée en Degré Minute Seconde  
**Altitude minimale/maximale** : hauteur minimum et maximum de la ville par rapport au niveau de l’eau  

In [19]:
columns=['ville_id','ville_departement','ville_slug','ville_nom','ville_nom_simple','ville_nom_reel',
           'ville_nom_soundex','ville_nom_metaphone','ville_code_postal','ville_commune','ville_code_commune',
           'ville_arrondissement','ville_canton','ville_amdi','ville_population_2010','ville_population_1999',
           'ville_population_2012','ville_densite_2010','ville_surface','ville_longitude_deg','ville_latitude_deg',
           'ville_longitude_grd','ville_latitude_grd','ville_longitude_dms','ville_latitude_dms','ville_zmin',
           'ville_zmax'] 

 
ville_df = pd.read_csv(os.path.join(DATA_DIR, GEO_FILENAME),sep=',', dtype={'ville_code_commune': str}, names=columns, header=None)

ville_df.head()

/Users/alexiscot/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ville_id,ville_departement,ville_slug,ville_nom,ville_nom_simple,ville_nom_reel,ville_nom_soundex,ville_nom_metaphone,ville_code_postal,ville_commune,...,ville_densite_2010,ville_surface,ville_longitude_deg,ville_latitude_deg,ville_longitude_grd,ville_latitude_grd,ville_longitude_dms,ville_latitude_dms,ville_zmin,ville_zmax
0,1,1,ozan,OZAN,ozan,Ozan,O250,OSN,01190,284,...,93,6.60,4.91667,46.3833,2866.0,51546.0,45456.0,462330.0,170.0,205.0
1,2,1,cormoranche-sur-saone,CORMORANCHE-SUR-SAONE,cormoranche sur saone,Cormoranche-sur-Saône,C65652625,KRMRNXSRSN,01290,123,...,107,9.85,4.83333,46.2333,2772.0,51379.0,44953.0,461427.0,168.0,211.0
2,3,1,plagne-01,PLAGNE,plagne,Plagne,P425,PLKN,01130,298,...,20,6.20,5.73333,46.1833,3769.0,51324.0,54342.0,461131.0,560.0,922.0
3,4,1,tossiat,TOSSIAT,tossiat,Tossiat,T230,TST,01250,422,...,138,10.17,5.31667,46.1333,3309.0,51268.0,51854.0,460828.0,244.0,501.0
4,5,1,pouillat,POUILLAT,pouillat,Pouillat,P430,PLT,01250,309,...,14,6.23,5.43333,46.3333,3435.0,51475.0,52542.0,461938.0,333.0,770.0


In [20]:
ville_df = ville_df[['ville_code_commune','ville_canton','ville_densite_2010','ville_longitude_deg','ville_latitude_deg']]

In [21]:
ville_df.head()

,ville_code_commune,ville_canton,ville_densite_2010,ville_longitude_deg,ville_latitude_deg
0,01284,26.0,93,4.91667,46.3833
1,01123,27.0,107,4.83333,46.2333
2,01298,3.0,20,5.73333,46.1833
3,01422,25.0,138,5.31667,46.1333
4,01309,33.0,14,5.43333,46.3333


In [22]:
df.head()

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,...,PPAT15,PPSOC15,PPFAM15,PPMINI15,PPLOGT15,PIMPOT15,D115,D915,RD15,_merge
0,01001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,01002,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,01004,1.0,4.0,21.0,35.0,9.0,15.0,25.0,8.0,32.0,...,9.5,6.7,2.7,2.0,2.0,-15.8,10398.571429,33801.200000,3.250562,both
3,01005,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,01007,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,4.0,...,8.1,4.0,2.4,0.8,0.8,-15.1,13620.666667,34606.923077,2.540766,both


## Merge

In [23]:
df_2 = (df.merge(ville_df, left_on = 'codecommuneetablissement', right_on = 'ville_code_commune',
                   how='left'))

In [24]:
df_2._merge.value_counts()

both          20404
right_only    15444
left_only        74
Name: _merge, dtype: int64

In [25]:
df_2.head()

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,...,PIMPOT15,D115,D915,RD15,_merge,ville_code_commune,ville_canton,ville_densite_2010,ville_longitude_deg,ville_latitude_deg
0,01001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,both,01001,10.0,49.0,4.91667,46.15
1,01002,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,both,01002,1.0,24.0,5.41667,46.00
2,01004,1.0,4.0,21.0,35.0,9.0,15.0,25.0,8.0,32.0,...,-15.8,10398.571429,33801.200000,3.250562,both,01004,1.0,562.0,5.35000,45.95
3,01005,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,...,NaN,NaN,NaN,NaN,both,01005,30.0,101.0,4.90000,46.00
4,01007,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,4.0,...,-15.1,13620.666667,34606.923077,2.540766,both,01007,1.0,70.0,5.35000,46.00


In [26]:
print('Shape full dataframe: ',df_2.shape)

Shape full dataframe:  (35922, 52)


## Calcul de distance (km)

In [27]:
#Calcul de distance (km) jusqu'à une ville présentant au moins un médecin 


from math import sin, cos, sqrt, atan2, radians, asin

def distance(latitude1, latitude2, longitude1, longitude2):
    R = 6371.0 # approximate radius of earth in km
    latitude1 = radians(latitude1)
    latitude2 = radians(latitude2)
    longitude1 = radians(longitude1)
    longitude2 = radians(longitude2)
    
    dist_lon = longitude2 - longitude1
    dist_lat = latitude2 - latitude1

    a = sin(dist_lat / 2)**2 + cos(latitude1) * cos(latitude2) * sin(dist_lon / 2)**2

    c = 2 * asin(sqrt(a)) 
    
    distance = R * c
    
    return distance

**Distance généraliste**

In [28]:
df_2['dist_generaliste']=0

destination_cities = [
    {
        'i': i,
        'lat': row['ville_latitude_deg'],
        'long': row['ville_longitude_deg'],
    }
    for i, row in df_2.iterrows()
    if row['generaliste'] >= 1]

for i in df_2.index:
    row = df_2.iloc[i, :]

    if row['generaliste'] == 0:
        target_distances = [
            {
                'destination_i': i,
                'distance': distance(
                    latitude1 = row['ville_latitude_deg'],
                    latitude2 = destination['lat'],
                    longitude1 = row['ville_longitude_deg'],
                    longitude2 = destination['long']),
            }
            for destination in destination_cities]
        
        elem = min(target_distances, key=lambda x: x['distance'])
        df_2.loc[i, 'dist_generaliste'] = elem['distance']

In [46]:
print('Il y a', len(df_2[df_2.dist_generaliste >10]),"villes à plus de 10km d'au moins un médecin généraliste")

Il y a 85 villes à plus de 10km d'au moins un médecin généraliste


**Distance hôpital**

In [30]:
df_2['dist_hopital']=0

destination_cities = [
    {
        'i': i,
        'lat': row['ville_latitude_deg'],
        'long': row['ville_longitude_deg'],
    }
    for i, row in df_2.iterrows()
    if row['hopital'] >= 1]

for i in df_2.index:
    row = df_2.iloc[i, :]

    if row['hopital'] == 0:
        target_distances = [
            {
                'destination_i': i,
                'distance': distance(
                    latitude1 = row['ville_latitude_deg'],
                    latitude2 = destination['lat'],
                    longitude1 = row['ville_longitude_deg'],
                    longitude2 = destination['long']),
            }
            for destination in destination_cities]
        
        elem = min(target_distances, key=lambda x: x['distance'])
        df_2.loc[i, 'dist_hopital'] = elem['distance']

In [45]:
print('Il y a', len(df_2[df_2.dist_hopital >10]),"villes à plus de 10km d'un hôpital")

Il y a 4288 villes à plus de 10km d'un hôpital


**Distance soins de base**

In [33]:
df_2['dist_soin_base']=0

destination_cities = [
    {
        'i': i,
        'lat': row['ville_latitude_deg'],
        'long': row['ville_longitude_deg'],
    }
    for i, row in df_2.iterrows()
    if row['soin_de_base'] >= 3]

for i in df_2.index:
    row = df_2.iloc[i, :]

    if row['soin_de_base'] <3 :
        target_distances = [
            {
                'destination_i': i,
                'distance': distance(
                    latitude1 = row['ville_latitude_deg'],
                    latitude2 = destination['lat'],
                    longitude1 = row['ville_longitude_deg'],
                    longitude2 = destination['long']),
            }
            for destination in destination_cities]
        
        elem = min(target_distances, key=lambda x: x['distance'])
        df_2.loc[i, 'dist_soin_base'] = elem['distance']

In [44]:
print('Il y a', len(df_2[df_2.dist_soin_base >10]),"villes à plus de 10km d'au moins 3 soins de base")

Il y a 1178 villes à plus de 10km d'au moins 3 soins de base


**Distance spécialistes réguliers**

In [35]:
df_2['dist_spe_regulier']=0

destination_cities = [
    {
        'i': i,
        'lat': row['ville_latitude_deg'],
        'long': row['ville_longitude_deg'],
    }
    for i, row in df_2.iterrows()
    if row['spe_regulier'] >= 3]

for i in df_2.index:
    row = df_2.iloc[i, :]

    if row['spe_regulier'] <3 :
        target_distances = [
            {
                'destination_i': i,
                'distance': distance(
                    latitude1 = row['ville_latitude_deg'],
                    latitude2 = destination['lat'],
                    longitude1 = row['ville_longitude_deg'],
                    longitude2 = destination['long']),
            }
            for destination in destination_cities]
        
        elem = min(target_distances, key=lambda x: x['distance'])
        df_2.loc[i, 'dist_spe_regulier'] = elem['distance']

In [43]:
print('Il y a', len(df_2[df_2.dist_spe_regulier > 10]),"villes à plus de 10km d'au moins 3 spécialistes réguliers")

Il y a 133 villes à plus de 10km d'au moins 3 spécialistes réguliers


**Distance autres spécialistes**

In [37]:
df_2['dist_spe_particulier']=0

destination_cities = [
    {
        'i': i,
        'lat': row['ville_latitude_deg'],
        'long': row['ville_longitude_deg'],
    }
    for i, row in df_2.iterrows()
    if row['spe_particulier'] >= 3]

for i in df_2.index:
    row = df_2.iloc[i, :]

    if row['spe_particulier'] <3 :
        target_distances = [
            {
                'destination_i': i,
                'distance': distance(
                    latitude1 = row['ville_latitude_deg'],
                    latitude2 = destination['lat'],
                    longitude1 = row['ville_longitude_deg'],
                    longitude2 = destination['long']),
            }
            for destination in destination_cities]
        
        elem = min(target_distances, key=lambda x: x['distance'])
        df_2.loc[i, 'dist_spe_particulier'] = elem['distance']

In [42]:
print('Il y a', len(df_2[df_2.dist_spe_regulier > 10]),"villes à plus de 10km d'au moins 3 spécialistes particuliers")

Il y a 133 villes à plus de 10km d'au moins 3 spécialistes particuliers


In [47]:
df_2.head()

,codecommuneetablissement,ambulance,analyse_medicale,autre,autre_specialiste,chirurgien,dentiste,generaliste,hopital,infirmiers,...,ville_code_commune,ville_canton,ville_densite_2010,ville_longitude_deg,ville_latitude_deg,dist_generaliste,dist_hopital,dist_soin_base,dist_spe_regulier,dist_spe_particulier
0,01001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,01001,10.0,49.0,4.91667,46.15,3.919269,5.343667,5.343667,3.851641,5.343667
1,01002,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,01002,1.0,24.0,5.41667,46.00,3.702791,7.579890,3.702791,3.702791,3.805566
2,01004,1.0,4.0,21.0,35.0,9.0,15.0,25.0,8.0,32.0,...,01004,1.0,562.0,5.35000,45.95,0.000000,0.000000,0.000000,0.000000,0.000000
3,01005,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,...,01005,30.0,101.0,4.90000,46.00,0.000000,9.822801,9.011880,0.000000,9.011880
4,01007,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,4.0,...,01007,1.0,70.0,5.35000,46.00,0.000000,5.559746,0.000000,0.000000,0.000000


## Export

In [39]:
df_2.to_csv(os.path.join(PROCESS_DIR, ANALYSIS_FILENAME), sep=';', index=False)